In [1]:
from yolov5 import detect
from PIL import Image
import os
from glob import glob
import torch

In [2]:
torch.manual_seed(777)
detected_train_formats = detect.object_detection(imgsz=[576], source="yolov5/hanssem/images/train_val")

YOLOv5 🚀 ca0ae5d torch 1.9.0+cu102 CUDA:0 (Tesla V100-PCIE-32GB, 32510.5MB)
                                    CUDA:1 (Tesla V100-PCIE-32GB, 32510.5MB)
                                    CUDA:2 (Tesla V100-PCIE-32GB, 32510.5MB)
                                    CUDA:3 (Tesla V100-PCIE-32GB, 32510.5MB)

Fusing layers... 
Model Summary: 224 layers, 7059304 parameters, 0 gradients, 16.3 GFLOPs


image 1/1814 /workspace/fourElse/KimHeeSu/2021-1-CECD3-four-elSe-1/integrated_main/yolov5/hanssem/images/train_val/100356.jpg: 576x576 1 closet, Done. (0.014s)
image 2/1814 /workspace/fourElse/KimHeeSu/2021-1-CECD3-four-elSe-1/integrated_main/yolov5/hanssem/images/train_val/100357.jpg: 576x576 1 closet, Done. (0.009s)
image 3/1814 /workspace/fourElse/KimHeeSu/2021-1-CECD3-four-elSe-1/integrated_main/yolov5/hanssem/images/train_val/102460.jpg: 576x576 1 closet, Done. (0.010s)
image 4/1814 /workspace/fourElse/KimHeeSu/2021-1-CECD3-four-elSe-1/integrated_main/yolov5/hanssem/images/train_val/102461.jpg: 576x576 3 closets, Done. (0.005s)
image 5/1814 /workspace/fourElse/KimHeeSu/2021-1-CECD3-four-elSe-1/integrated_main/yolov5/hanssem/images/train_val/102463.jpg: 576x576 1 closet, Done. (0.006s)
image 6/1814 /workspace/fourElse/KimHeeSu/2021-1-CECD3-four-elSe-1/integrated_main/yolov5/hanssem/images/train_val/102471.jpg: 576x576 1 closet, 1 bed, Done. (0.006s)
image 7/1814 /workspace/fourElse

In [5]:
import pickle


In [6]:
with open("detected_train_formats.p", "wb") as out:
    pickle.dump(detected_train_formats, out)
    

In [7]:

with open("./detected_train_formats.p","rb") as f:
    detected_train_formats = pickle.load(f)

from glob import glob
image_path = [format["objectImagePath"] for format in detected_train_formats]
detected_images_train = [Image.open(path) for path in image_path]

In [8]:
detected_train_formats[0]

{'objectID': '0336cb24-0b8f-4f18-ae64-ef21ce42cd10',
 'location': {'xmin': 76, 'ymin': 170, 'xmax': 472, 'ymax': 537},
 'tag': 'closet',
 'objectImagePath': 'yolov5/runs/detect/exp11/croppedImages/100356_closet.jpg'}

In [9]:
from ImageRetrievalClass import ImageRetrievalClass

retrievalInstance = ImageRetrievalClass("IncepResNet", True, False)

Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


In [10]:
retrievalInstance.readTrainSet(detected_images_train)

Reading train images
train image shape = (512, 512, 3)


In [11]:
retrievalInstance.buildModel()

Loading IncepResNet pre-trained model...
219062272/219055592 [==============================] - 9s 0us/step
Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 255, 255, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 255, 255, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 255, 255, 32) 0           batch_

In [12]:
X_train = retrievalInstance.trainTransform()

Applying image transformer to training images...
 -> X_train.shape = (2312, 512, 512, 3)


In [14]:
import numpy as np
np.save("./X_train.npy",X_train)

In [15]:
E_train = retrievalInstance.predictTrain(X_train) # 연결이 끊기네... (256, 256)으로 바꾸니 해결됨

Inferencing embeddings using pre-trained model...
 -> E_train.shape = (2312, 14, 14, 1536)


# 객체 각각의 npy 파일 저장

In [18]:
detected_train_formats[0]["objectImagePath"][39:-4]

'100356_closet'

In [19]:
for idx, format in enumerate(detected_train_formats):
    filename = format["objectImagePath"][39:-4]
    npyFilename = "./npyFiles/"+filename+".npy"
    np.save(npyFilename, E_train[idx])


# JSON 파일 저장

In [20]:
import json
for idx, format in enumerate(detected_train_formats):
    npyFilename = "./npyFiles/"+format["objectImagePath"][39:-4]+".npy"
    jsonDict = {
        "objectID":format["objectID"],
        "location":format["location"],
        "tag":format["tag"],
        "objectImagePath" : format["objectImagePath"],
        "npyPath": npyFilename
    }

    filename = "./jsonFiles/"+format["objectImagePath"][39:-4]+".json"

    with open(filename, "w") as outFile:
        json.dump(jsonDict, outFile, indent=2)